In [17]:
%load_ext autoreload
%autoreload 2

import os, sys, re
from pathlib import Path

pj_dir = Path(os.getcwd()).parent
data_dir = pj_dir/'data'
src_dir = pj_dir/'src'
sys.path.append(str(src_dir))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from requests_html import HTMLSession
import pandas as pd

In [3]:
sess = HTMLSession()

In [4]:
r = sess.get('http://www-shibuya.jp/schedule/#wwwxwww/201806')

In [5]:
articles = r.html.find('#eventList > div > article')

In [8]:
from datetime import date
class Event():
    def __init__(self):
        self.year = 0
        self.month = 0
        self.day = 0
        self.title = ''
        self.artist = ''
        self.description = ''
        self.url = ''
        self.open_ = ''
        self.start = ''
        self.place = ''
        
    @property
    def date(self):
        return date(self.year, self.month, self.day)
    
    def tolist(self):
        return [self.place, self.title, self.artist, self.description, self.url, self.date, self.open_, self.start]

In [14]:
ls_events = []
for article in articles:
    event = Event()
    event.url = article.find('.pageLink', first=True).attrs.get('href', '')
    event.year = 2018
    event.month = 6
    event.day = int(article.find('.inner .event .date .day', first=True).text)
    event.place = article.find('.inner .placeLabel span', first=True).text
    event.artist = article.find('.inner .info .title', first=True).text
    event.title = article.find('.inner .info .exp span', first=True).text
    openstart = article.find('.inner .info .openstart', first=True).text
    oss = openstart.replace('OPEN / START\u3000', '').split('/')
    if len(oss) == 2:
        event.open_, event.start = oss
    ls_events.append(event)

In [20]:
os.listdir(data_dir/'live')

['201806_www']

In [21]:
df = pd.DataFrame([e.tolist() for e in ls_events], columns=['place', 'title', 'artist', 'description', 'url', 'date', 'open_', 'start'])
df.to_csv(data_dir/'live/201806_www.csv')